# Introduction
 In this assignment, you will do exercises on arcpy.Exists(), arcpy.Walk(), arcpy.da.Describe(), python list comprehension, arcpy.da.SearchCursor, and arcpy.da.InsertCursor(). In each section, a code block is provided but not ready to run. You need to add this notebook to ArcGIS Pro, modify each block to make it runable, keep the output message , and write an explanation of the code block. 

 Data preparation: 
 Use the same zip file from assignment 4 for this exercise. The zip file contains a geodatabase and a folder with shapefles. Download the data from the [data](../data) folder of the github website. Extract the zip file and use the geodatabase and the folder containing the shapefiles accordingly.  

## Read a text file for data and write them into a feature class using InsertCursor

- Use the points.csv file in the same folder of the notebook 
- The code block can read the csv file and write its contents to a feature class
- The spatial reference of the feature class is defined as GCS WGS84 (code 4326) according to the coordinates in the csv file

In [ ]:
import arcpy
import csv

# Prompt for user input
csv_file_path = "Enter the path to the CSV file: "
geodatabase_path = "Enter the path to the geodatabase: "
feature_class_name = "Enter the name of the new feature class: "

# Determine the number of fields and field names from the CSV header
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    num_fields = len(header)

# Create a new point feature class
sr = arcpy.SpatialReference(4326)  # You can specify the appropriate spatial reference
arcpy.CreateFeatureclass_management(geodatabase_path, feature_class_name, "POINT", spatial_reference=sr)

# Add fields to the feature class based on the CSV header
field_info = arcpy.ListFields(geodatabase_path + "/" + feature_class_name)
field_names = [field.name for field in field_info]
for field in header:
    if field not in field_names:
        arcpy.AddField_management(geodatabase_path + "/" + feature_class_name, field, "DOUBLE")

# Create a cursor for inserting point features with additional fields
cursor_fields = ["SHAPE@X", "SHAPE@Y"] + header
with arcpy.da.InsertCursor(geodatabase_path + "/" + feature_class_name, cursor_fields) as cursor:
    # Read data from the CSV file and create point features with attributes
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Skip the header row (if present)
        next(csv_reader, None)
        
        for row in csv_reader:
            x = float(row[0])
            y = float(row[1])
            additional_fields = row
            cursor.insertRow((x, y) + tuple(additional_fields))

print(f"New point feature class '{feature_class_name}' created in '{geodatabase_path}' with {num_fields} fields.")



## Use arcpy.Exist()

- Write a Python script that checks the existence of a specified dataset within an ArcGIS workspace.
- Prompt the user for input (e.g., dataset name and workspace path).
- Use arcpy.Exists() to check if the dataset exists.
- Print a message indicating whether the dataset exists or not.

In [ ]:
import arcpy
import os

# Prompt for user input
dataset_name = "Enter the dataset name: "
workspace_path = "Enter the workspace path: "
fullname = os.path.join(workspace_path,dataset_name)
# Check if the dataset exists
if arcpy.Exists(fullname):
    print(f"The dataset '{dataset_name}' exists in the workspace.")
else:
    print(f"The dataset '{dataset_name}' does not exist in the workspace.")

**Enter your explanation of the above code block by editing this markdown block**



## Use arcpy.Walk()
- Use arcpy.Walk() to iterate through all feature datasets in a specified geodatabase.
- List all the feature classes within each dataset.
- Display the results.

In [ ]:
import arcpy

# Define the workspace
workspace = "C:/path_to_geodatabase.gdb"

# Use arcpy.Walk() to iterate through feature datasets
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace, datatype="FeatureClass"):
    for filename in filenames:
        print(f"Feature Class in {dirpath}: {filename}")

**Enter your explanation of the above code block by editing this markdown block**



## Use List Comprehension

- Write a script that uses list comprehension to generate a list of .shp files in a specified folder.
- Print the list of file names


In [ ]:
import os

# Specify the folder path
folder_path = "C:/path_to_folder"

# Use list comprehension to generate a list of .shp files
shp_files = [file for file in os.listdir(folder_path) if file.endswith(".shp")]

# Print the list of .shp files
print("Shapefiles in the folder:")
for shp_file in shp_files:
    print(shp_file)

## Use arcpy.da.SearchCursor

- Build a script that uses arcpy.da.SearchCursor to extract attribute information from a feature class.
- Prompt the user for the feature class path and the field(s) to extract.
- Use arcpy.da.SearchCursor to extract the data.
- Display the extracted data.


In [ ]:
import arcpy

# Prompt for user input
fc_path = "Enter the feature class path: "
fields_to_extract = ["Enter the field(s) to extract (comma-separated): "]

# Use arcpy.da.SearchCursor to extract data
with arcpy.da.SearchCursor(fc_path, fields_to_extract) as cursor:
    print("Extracted Data:")
    for row in cursor:
        print([row[i] for i in range(len(fields_to_extract)])

## Use addField and field Calculator

- Add a new field for which you want to perform the calculation.

- Uses the Field Calculator tool in the script with the arcpy.CalculateField_management() method to calculate a new field based on a specified expression. The expression can be a simple calculation, text manipulation, or a more complex conditional expression.

- Specifies the expression using Python string formatting and ensures that it follows the field calculator syntax

In [ ]:
import arcpy
fc = "feature class name"
newfieldName = "newfieldname"
arcpy.AddField_management(fc, newFieldName, "SHORT")
expression = "reclass(!Field1!)"
codeblock = """
def reclass(field):
    if (field > 10):
        return 1
    elif (field > 20):
        return 2
    else:
        return 0"""
arcpy.CalculateField_management(fc, newFieldName, expression, "", codeblock)